In [12]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://m.realtytrac.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://m.realtytrac.com/mapsearch/sold/ca/')

True

# Thông tin nhóm

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# Câu hỏi

Cho các thông tin về căn nhà:

- Giá cho thuê trước khi bán.
- Địa chỉ.
- Loại nhà.
- Số phòng ngủ.
- Số nhà vệ sinh.
- Diện tích phần nhà.
- Diện tích toàn bộ lô đất.
- Năm xây dựng.
- Tiền thuế.
- Gần mấy trường học.
- Giá được bán gần đây nhất.

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

Dữ liệu thu thập từ trang web https://www.realtytrac.com/. Ta chỉ thu thập dữ liệu "các estate đã được bán tại California", ta có HTML cần parse: https://m.realtytrac.com/mapsearch/sold/ca/

### Start

In [6]:
# Import các thư viện cần thiết
import urllib.robotparser
import json
import os.path
import time
import traceback
import re
from datetime import datetime
from pprint import pprint

from requests_html import HTMLSession

In [2]:
session = HTMLSession()
r = session.get('https://m.realtytrac.com/mapsearch/sold/ca/')

# Get list url Countries of California
base_url = "https://m.realtytrac.com"
list_url = []

counties = r.html.find("option")
for county in counties:
    tail_url = county.attrs['value']
    if (tail_url):
        url = base_url + tail_url
        list_url.append(url)

list_url

['https://m.realtytrac.com/mapsearch/sold/ca/alameda-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/alpine-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/amador-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/butte-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/calaveras-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/colusa-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/contra-costa-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/del-norte-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/el-dorado-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/fresno-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/glenn-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/humboldt-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/imperial-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/inyo-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/kern-county/',
 'https://m.realtytrac.com/mapsearch/sold/ca/kings-count

In [30]:
# Hàm trả về generator, generator này trả về trang tiếp theo sau mỗi lần duyệt
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p-{i}"
        i += 1
        
# Hàm parse để lấy thông tin chi tiết của estate từ detail Url của estate đã bán.      
def parseDetailPage(session: HTMLSession, url: str):
    #global logger
    #logger.info(f"Processing detail url: {url}")
    
    # Lấy mã HTML của trang web
    r = session.get(url)
    
    # Dictionary lưu lại kết quả
    result = {}
    
    # Lấy addrass của estate
    address = {}
    street = r.html.find("[itemprop = 'streetAddress']", first=True)
    if street is not None:
        street = street.text
        
    locality = r.html.find("[itemprop = 'addressLocality']", first=True)
    if locality is not None:
        locality = locality.text
        
    region = r.html.find("[itemprop = 'addressRegion']", first=True)
    if region is not None:
        region = region.text
    
    code = r.html.find("[itemprop = 'postalCode']", first=True)
    if code is not None:
        code = code.text
    
    address["street"] = street
    address["locality"] = locality
    address["region"] = region
    address["code"] = code
    
    result["address"] = address
    
    # Lấy thông tin ngày đã bán estate
    date_sold = r.html.find(".recently-sold", first=True)
    if date_sold is not None:
        result["date_sold"] = date_sold.text.replace("SOLD ON ", "")
    else:
        result["date_sold"] = None
    
    
    # Lấy giá cho thuê trước khi bán
    mortgage = r.html.find(".property-price-info", first=True)
    if mortgage is not None:
        result["mortgage"] = mortgage.text.replace("Est. Mortgage: ","")
    else:
        result["mortgage"] = None
    
    
    details = []
    detailTexts = r.html.find(".detail-text")
    if detailTexts is not None:
        for detailText in detailTexts:
            details.append(detailText.html)
    result["details"] = details
    
    # Lấy property info của estate
    propertyInfo = r.html.find(".property-info", first=True)
    if propertyInfo is not None:
        info = {}
        items = propertyInfo.find(".item")
        if items is not None:
            for item in items:
                name = item.find(".name", first=True).text.lower().replace(" ", "_")
                value = item.find(".value", first=True).text
                info[name] = value

        result["info"] = info
    else:
        result["info"] = None

    property_taxes = r.html.find(".property-taxes")
    if property_taxes is not None:
        for tax in property_taxes:
            key = tax.find(".section-head", first=True).text.lower().replace(" ", "_")
            taxes = {}
            items = tax.find(".item")
            if items is not None:
                for item in items:
                    name = item.find(".name", first=True).text.lower().replace(" ", "_")
                    value = item.find(".value", first=True).text
                    taxes[name] = value

            result[key] = taxes
  
    # Lấy số lượng trường học gần đó
    local_school = r.html.find(".property-schools", first=True)
    num_school = 0
    if local_school is not None:
        num_school = len(local_school.find(".schoolInfo"))
        
    result["school"] = num_school
    
    # Lấy thông tin tội phạm trong vùng
    crimes = {}
    local_crime_index = r.html.find(".property-local-crime", first=True)
    if local_crime_index is not None:
        type_crimes = local_crime_index.find(".gradeTitle")
        if type_crimes is not None:
            for type_crime in type_crimes:
                match = re.match(r"(.+?) = (\d+)", type_crime.text)
                if match is not None:
                    key = match.group(1).lower().replace(' ', '_')
                    value = int(match.group(2))
                    crimes[key] = value
                    
    result["local_crime_index"] = crimes

    # Lấy số lượng nhà bị tịch thu gần đó
    nearby_foreclosures = r.html.find(".property-nearby-fc", first=True)
    num_fc = 0
    if nearby_foreclosures is not None:
        num_fc = len(nearby_foreclosures.find(".nearby-property"))

    result["foreclosures"] = num_fc
    
    return result


In [31]:
def parseSearchPage(session, url):
    r = session.get(url)

    detailUrls = [link for link in r.html.absolute_links if 'property' in link]
    
    results = []
    for detailUrl in detailUrls:
        if not rp.can_fetch('*', detailUrl):
            print(f"SKIP: {detailUrl}")
            continue
        print(f"Parsing detail url {detailUrl}")
        result = parseDetailPage(session, detailUrl)
        results.append(result)
        
    nextPageAnchor = r.html.find(".current + .page", first=True)
    return results, nextPageAnchor is not None

In [32]:
def getAllCountyProperties(session, countyUrl):
    if countyUrl.endswith("/"):
        countyUrl = countyUrl[:-1]
    fileName = f"data/{os.path.basename(countyUrl)}.json"
    
    if os.path.exists(fileName):
        print(f"FILE EXISTS: {fileName}")
        return
    
    with open(fileName, "a") as fOut:
        for url in urlGenerator(countyUrl, 1):
            if not rp.can_fetch('*', url):
                print(f"SKIP: {url}")
                continue
            print(f"Parsing {url}")
            results, nextPage = parseSearchPage(session, url)
            for result in results:
                json.dump(result, fOut, ensure_ascii=False)
                fOut.write("\n")
            if not nextPage:
                break

In [33]:
url = "https://m.realtytrac.com/mapsearch/sold/ca/alameda-county"
session = HTMLSession()
getAllCountyProperties(session, url)

Parsing https://m.realtytrac.com/mapsearch/sold/ca/alameda-county
Parsing detail url https://m.realtytrac.com/property/ca/livermore/94551/1527-marigold-rd/155704592/
Parsing detail url https://m.realtytrac.com/property/ca/kensington/94707/26-highgate-rd/156554420/
Parsing detail url https://m.realtytrac.com/property/ca/hayward/94542/28630-barn-rock-dr/24319992/
Parsing detail url https://m.realtytrac.com/property/ca/fremont/94538/2875-wilson-cmn/17364498/
Parsing detail url https://m.realtytrac.com/property/ca/livermore/94551/6452-aspenwood-way/29399893/
Parsing detail url https://m.realtytrac.com/property/ca/kensington/94708/233-purdue-ave/157292061/
Parsing detail url https://m.realtytrac.com/property/ca/pleasanton/94566/1648-ramblewood-way/158047832/
Parsing detail url https://m.realtytrac.com/property/ca/kensington/94708/purdue-ave/241194953/
Parsing detail url https://m.realtytrac.com/property/ca/livermore/94550/1651-4th-st/53546762/
Parsing detail url https://m.realtytrac.com/pro

In [64]:
import json
import os.path

def xstr(s):
    return '' if s is None else str(s)

def preprocess(inputFile : str, outputFile : str):
    if os.path.exists(outputFile):
        print(f"FILE EXISTS: {outputFile}")
        return
    
    if not os.path.exists(inputFile):
        print(f"Input file not found: {inputFile}")
        return
    
    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "address_street",
            "address_locality",
            "address_region",
            "address_code",
            "date_sold",
            "mortgage",
            "info_type",
            "info_bedrooms",
            "info_bathrooms",
            "info_size",
            "info_lot_size",
            "info_year_built",
            "info_est_value",
            "info_sold_price",
            "info_property_id",
            "info_county",
            "info_parcel_number",
            "taxes_land",
            "taxes_improvements",
            "taxes_total",
            "taxes_taxes",
            "school",
            "total_crime",
            "violent_crime",
            "property_crime",
            "foreclosures",
        )
        
        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t"
        fOut.write(f"{tab.join(headers)}\n")
        
        for line in fIn:
            row = json.loads(line)
            
            # Lấy địa chỉ
            address = row.get("address")
            address_street = address.get("street")
            address_locality = address.get("locality")
            address_region = address.get("region")
            address_code = address.get("code")
            
            # Lấy ngày bán thành công
            date_sold = row.get("date_sold")
            
            # Lấy giá cho thuê mỗi tháng (trước khi được bán)
            mortgage = row.get("mortgage")
            if mortgage is not None:
                mortgage = float(mortgage.replace("$","").replace(",","").replace("/mo","").replace("Est. Refinance: ", ""))
            
            # Lấy phần info
            info = row.get("info")
            
            # Type
            info_type = info.get("type")
            
            # Bedrooms
            info_bedrooms = info.get("bedrooms")
            if info_bedrooms  == "Contact Agent":
                info_bedrooms = None
            else:
                info_bedrooms = float(info_bedrooms)
            
            # Bathrooms
            info_bathrooms = info.get("bathrooms")
            if info_bathrooms == "Contact Agent":
                info_bathrooms = None
            else:
                info_bathrooms = float(info_bathrooms)
            
            # Size
            info_size = info.get("size")
            if info_size  == "Contact Agent":
                info_size = None
            else:
                info_size = float(info_size.replace(",","").replace(" sqft", ""))
            
            # Lot size
            info_lot_size = info.get("lot_size")
            if info_lot_size  == "Contact Agent":
                info_lot_size = None
            else:
                info_lot_size = float(info_lot_size.replace(",","").replace(" sqft", "").replace(" acres", ""))
            
            # Year build
            info_year_built = info.get("year_built")
            if info_year_built  == "Contact Agent":
                info_year_built = None
            else:
                info_year_built = int(info_year_built)
        
            # Est value
            info_est_value = info.get("est._value")
            if info_est_value is not None:
                info_est_value = float(info_est_value.replace("$","").replace(",",""))
            
            # Sold price
            info_sold_price = info.get("sold_price")
            if info_sold_price is not None:
                if info_sold_price != "N/A":
                    info_sold_price = float(info_sold_price.replace("$","").replace(",",""))
            
            # ID
            info_property_id = info.get("property_id")
            
            # County
            info_county = info.get("county")
            
            # Parcel_number
            info_parcel_number = info.get("parcel_number")
            
            # Lấy phần Taxes
            taxes = row.get("property_taxes")
            taxes_land = None
            taxes_improvements = None
            taxes_total = None
            taxes_taxes = None
            if taxes is not None:
                taxes_land = float(taxes.get("land").replace("$","").replace(",",""))
                taxes_improvements = float(taxes.get("improvements").replace("$","").replace(",",""))
                taxes_total = float(taxes.get("total").replace("$","").replace(",",""))
                taxes_taxes = taxes.get("taxes").replace("$","").replace(",","")
           
            # Lấy số lượng scholl gần đó
            school = int(row.get("school"))
            
            # Lấy tình hình tội phạm
            crime = row.get("local_crime_index")
            total_crime = int(crime.get("total_crime"))
            violent_crime = int(crime.get("violent_crime"))
            property_crime = int(crime.get("property_crime"))
            
            # Lấy số lượng foreclosures gần đó
            foreclosures = int(row.get("foreclosures"))
        
        
            row = (
                xstr(address_street),
                xstr(address_locality),
                xstr(address_region),
                xstr(address_code),
                xstr(date_sold),
                xstr(mortgage),
                xstr(info_type),
                xstr(info_bedrooms),
                xstr(info_bathrooms),
                xstr(info_size),
                xstr(info_lot_size),
                xstr(info_year_built),
                xstr(info_est_value),
                xstr(info_sold_price),
                xstr(info_property_id),
                xstr(info_county),
                xstr(info_parcel_number),
                xstr(taxes_land),
                xstr(taxes_improvements),
                xstr(taxes_total),
                xstr(taxes_taxes),
                xstr(school),
                xstr(total_crime),
                xstr(violent_crime),
                xstr(property_crime),
                xstr(foreclosures),
            )
            
            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

In [65]:
inputFile = "data/alameda-county.json"
outputFile = "data/alameda-county.csv"
preprocess(inputFile, outputFile)

In [53]:
import pandas as pd

In [66]:
file_csv = "data/alameda-county.csv"
data_df =  pd.read_csv(file_csv, sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,1527 Marigold Rd,Livermore,CA,94551,11/06/2019,2089.0,Single Family Residence,2.0,2.0,866.0,...,099 003702900,28021.0,38985.0,67006.0,1427 (2.12 %),3,51,33,92,4
1,26 Highgate Rd,Kensington,CA,94707,07/10/1969,4143.0,Single Family Residence,3.0,1.5,1548.0,...,5722310140,25832.0,44418.0,70250.0,2368 (3.37 %),3,25,16,47,4
2,28630 Barn Rock Dr,Hayward,CA,94542,11/06/2019,5360.0,Single Family Residence,4.0,2.5,2962.0,...,425 049001800,384564.0,325599.0,710163.0,9439 (1.32 %),3,123,84,213,4
3,2875 Wilson Cmn,Fremont,CA,94538,12/09/1983,3647.0,Townhouse,3.0,2.5,1748.0,...,525 078510600,46132.0,200474.0,246606.0,3335 (1.35 %),3,59,40,103,4
4,6452 Aspenwood Way,Livermore,CA,94551,11/06/2019,3820.0,Single Family Residence,4.0,2.5,1882.0,...,099B535303900,161822.0,377585.0,539407.0,7247 (1.34 %),3,51,33,92,4
5,233 Purdue Ave,Kensington,CA,94708,11/08/2019,3906.0,Single Family Residence,2.0,1.0,1186.0,...,5701910076,46675.0,57845.0,104520.0,2624 (2.51 %),3,25,16,47,4
6,1648 Ramblewood Way,Pleasanton,CA,94566,02/04/1965,4701.0,Single Family Residence,3.0,2.0,1675.0,...,946 337801200,28306.0,63431.0,91737.0,1067 (1.16 %),0,33,19,65,0
7,Purdue Ave,Kensington,CA,94708,11/08/2019,3906.0,MISCELLANEOUS,NaN,NaN,NaN,...,5701910084,2008.0,0.0,2008.0,116 (5.80 %),0,25,16,47,2
8,1651 4th St,Livermore,CA,94550,11/06/2019,3225.0,Single Family Residence,2.0,1.0,1046.0,...,097 004100504,131287.0,248003.0,379290.0,5297 (1.39 %),3,51,33,92,4
9,190 Stanford Ave,Kensington,CA,94708,11/18/2019,5215.0,Single Family Residence,2.0,1.0,1296.0,...,5700120024,234313.0,180953.0,415266.0,6548 (1.57 %),3,25,16,47,4


In [67]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9593 entries, 0 to 9592
Data columns (total 26 columns):
address_street        9593 non-null object
address_locality      9593 non-null object
address_region        9593 non-null object
address_code          9593 non-null int64
date_sold             9442 non-null object
mortgage              9481 non-null float64
info_type             9593 non-null object
info_bedrooms         7959 non-null float64
info_bathrooms        8112 non-null float64
info_size             8503 non-null float64
info_lot_size         8259 non-null float64
info_year_built       8472 non-null float64
info_est_value        822 non-null float64
info_sold_price       8659 non-null float64
info_property_id      9442 non-null float64
info_county           9593 non-null object
info_parcel_number    9593 non-null object
taxes_land            8892 non-null float64
taxes_improvements    8892 non-null float64
taxes_total           8892 non-null float64
taxes_taxes           8

### End